>[Rebalancing](#updateTitle=true&folderId=1Vf62KS3fdi5XEz3Eibgj40ZsrrKTVRfI&scrollTo=gORaPtwjq5d6)

>>[Random Forest](#updateTitle=true&folderId=1Vf62KS3fdi5XEz3Eibgj40ZsrrKTVRfI&scrollTo=NRRtCIPUzHwb)

>>[SVM](#updateTitle=true&folderId=1Vf62KS3fdi5XEz3Eibgj40ZsrrKTVRfI&scrollTo=MekvhT9uzPDD)

>>[Naive Bayes](#updateTitle=true&folderId=1Vf62KS3fdi5XEz3Eibgj40ZsrrKTVRfI&scrollTo=8IGWMNN602Qz)

>[Sample Weights](#updateTitle=true&folderId=1Vf62KS3fdi5XEz3Eibgj40ZsrrKTVRfI&scrollTo=hzNRVDfu1exm)

>>[Random Forest](#updateTitle=true&folderId=1Vf62KS3fdi5XEz3Eibgj40ZsrrKTVRfI&scrollTo=6296FJdf2szo)

>>[SVM](#updateTitle=true&folderId=1Vf62KS3fdi5XEz3Eibgj40ZsrrKTVRfI&scrollTo=slNmL1gn213a)

>>[Naive Bayes](#updateTitle=true&folderId=1Vf62KS3fdi5XEz3Eibgj40ZsrrKTVRfI&scrollTo=Bztd07ER3BjK)

>[Minimizing Expected Cost](#updateTitle=true&folderId=1Vf62KS3fdi5XEz3Eibgj40ZsrrKTVRfI&scrollTo=23amfjZBeSSW)

>>[Random Forest](#updateTitle=true&folderId=1Vf62KS3fdi5XEz3Eibgj40ZsrrKTVRfI&scrollTo=o_GcmjqHeVNF)

>>[SVM](#updateTitle=true&folderId=1Vf62KS3fdi5XEz3Eibgj40ZsrrKTVRfI&scrollTo=_g0cVh7meYC2)

>>[Naive Bayes](#updateTitle=true&folderId=1Vf62KS3fdi5XEz3Eibgj40ZsrrKTVRfI&scrollTo=Xc30xyX_eaRj)



The project is divided by methods (sampling, weighing and minimizing expected cost) and within each method we use three classifiers (Random Forest, SVM and Naive Bayes). At the end of each method, we are giving some conclusions about all three classifiers that were tested with this technique.

Note on how the cost matrix is defined in our code:

In the Statlog (German Credit Data) Data Set the labels are defined as:
*(1 = Good, 2 = Bad)*.

*The rows represent the actual classification and the columns the predicted classification*, while in general we use the transposed version.
**It is worse to class a customer as good when they are bad (5), than it is to class a customer as bad when they are good (1).**

This means that the cost matrix here is defined as: cost_m = [[0,1],[5,0]] but if we transpose it we get: cost_m = [[0,5],[1,0]], indicating again that the costs of a false positive (incorrectly saying an applicant is a good credit risk) is higher than the cost of a false negative (incorrectly saying an applicant is a bad credit risk).

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.utils import compute_sample_weight
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from joblib import dump, load
from collections import Counter
from sklearn.calibration import CalibratedClassifierCV
import warnings
warnings.filterwarnings('ignore')


In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data-numeric"
data = pd.read_csv(url, header=None, sep='\s+')

# define target column and create a DataFrame
target_col = 24
target_names = ['good', 'bad']
frame = pd.DataFrame(data[target_col], columns=['target'])
frame['target'].value_counts(sort=False)

Series([], Name: target, dtype: int64)

In [41]:
# split the data into features and labels
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print(Counter(y_train))
# define the cost matrix
cost_m = np.array([[0, 1], [5, 0]])

Counter({1: 486, 2: 214})


#Rebalancing

##Random Forest

In [42]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from collections import Counter


clf = RandomForestClassifier(n_estimators=100, random_state=0)
print("without sampling")
print(Counter(y_train))
#1: 486, 2: 214

model = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T # transpose to align with slides
print(conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)

print("with undersampling")
sampler = RandomUnderSampler(sampling_strategy={1: 486, 2: 114}, random_state=1) 
X_rs, y_rs = sampler.fit_resample(X_train, y_train)
print(Counter(y_rs))

model = clf.fit(X_rs, y_rs)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T # transpose to align with slides
print(conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)

print("with oversampling")
sampler = RandomOverSampler(sampling_strategy={1: 586, 2: 214}, random_state=1) 
X_rs, y_rs = sampler.fit_resample(X_train, y_train)
print(Counter(y_rs))

model = clf.fit(X_rs, y_rs)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T 
print(conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)

print("with combination")
sampler = RandomUnderSampler(sampling_strategy={1: 486, 2: 114}, random_state=1)
X_rs, y_rs = sampler.fit_resample(X_train, y_train)
sampler = RandomOverSampler(sampling_strategy={1: 586, 2: 114}, random_state=1)
X_rs, y_rs = sampler.fit_resample(X_rs, y_rs)
print(Counter(y_rs))

model = clf.fit(X_rs, y_rs)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T # transpose to align with slides
print(conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)

without sampling
Counter({1: 486, 2: 214})
              precision    recall  f1-score   support

        good       0.79      0.89      0.84       214
         bad       0.61      0.43      0.50        86

    accuracy                           0.76       300
   macro avg       0.70      0.66      0.67       300
weighted avg       0.74      0.76      0.74       300

[[190  49]
 [ 24  37]]
169

with undersampling
Counter({1: 486, 2: 114})
              precision    recall  f1-score   support

        good       0.75      0.96      0.85       214
         bad       0.70      0.22      0.34        86

    accuracy                           0.75       300
   macro avg       0.73      0.59      0.59       300
weighted avg       0.74      0.75      0.70       300

[[206  67]
 [  8  19]]
107

with oversampling
Counter({1: 586, 2: 214})
              precision    recall  f1-score   support

        good       0.78      0.90      0.83       214
         bad       0.58      0.36      0.45      

##SVM

In [43]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

clf = SVC(kernel='linear')
print("without sampling")
print(Counter(y_train))
#1: 486, 2: 214

model = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T # transpose to align with slides
print(conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)

print("with undersampling")
sampler = RandomUnderSampler(sampling_strategy={1: 486, 2: 114}, random_state=1) 
X_rs, y_rs = sampler.fit_resample(X_train, y_train)
print(Counter(y_rs))

model = clf.fit(X_rs, y_rs)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T # transpose to align with slides
print(conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)

print("with oversampling")
sampler = RandomOverSampler(sampling_strategy={1: 586, 2: 214}, random_state=1) 
X_rs, y_rs = sampler.fit_resample(X_train, y_train)
print(Counter(y_rs))

model = clf.fit(X_rs, y_rs)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T 
print(conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)

print("with combination")
sampler = RandomUnderSampler(sampling_strategy={1: 486, 2: 114}, random_state=1)
X_rs, y_rs = sampler.fit_resample(X_train, y_train)
sampler = RandomOverSampler(sampling_strategy={1: 586, 2: 114}, random_state=1)
X_rs, y_rs = sampler.fit_resample(X_rs, y_rs)
print(Counter(y_rs))

model = clf.fit(X_rs, y_rs)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T 
print(conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)

without sampling
Counter({1: 486, 2: 214})
              precision    recall  f1-score   support

        good       0.82      0.87      0.85       214
         bad       0.62      0.53      0.57        86

    accuracy                           0.77       300
   macro avg       0.72      0.70      0.71       300
weighted avg       0.77      0.77      0.77       300

[[186  40]
 [ 28  46]]
180

with undersampling
Counter({1: 486, 2: 114})
              precision    recall  f1-score   support

        good       0.76      0.94      0.84       214
         bad       0.66      0.27      0.38        86

    accuracy                           0.75       300
   macro avg       0.71      0.61      0.61       300
weighted avg       0.73      0.75      0.71       300

[[202  63]
 [ 12  23]]
123

with oversampling
Counter({1: 586, 2: 214})
              precision    recall  f1-score   support

        good       0.78      0.89      0.83       214
         bad       0.57      0.37      0.45      

##Naive Bayes

In [44]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

clf = GaussianNB()
print("without sampling")
print(Counter(y_train))
#1: 486, 2: 214

model = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T 
print(conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)

print("with undersampling")
sampler = RandomUnderSampler(sampling_strategy={1: 486, 2: 114}, random_state=1) 
X_rs, y_rs = sampler.fit_resample(X_train, y_train)
print(Counter(y_rs))

model = clf.fit(X_rs, y_rs)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T
print(conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)

print("with oversampling")
sampler = RandomOverSampler(sampling_strategy={1: 586, 2: 214}, random_state=1) 
X_rs, y_rs = sampler.fit_resample(X_train, y_train)
print(Counter(y_rs))

model = clf.fit(X_rs, y_rs)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T 
print(conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)

print("with combination")
sampler = RandomUnderSampler(sampling_strategy={1: 486, 2: 114}, random_state=1)
X_rs, y_rs = sampler.fit_resample(X_train, y_train)
sampler = RandomOverSampler(sampling_strategy={1: 586, 2: 114}, random_state=1)
X_rs, y_rs = sampler.fit_resample(X_rs, y_rs)
print(Counter(y_rs))


model = clf.fit(X_rs, y_rs)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T
print(conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)


without sampling
Counter({1: 486, 2: 214})
              precision    recall  f1-score   support

        good       0.84      0.73      0.78       214
         bad       0.50      0.65      0.56        86

    accuracy                           0.71       300
   macro avg       0.67      0.69      0.67       300
weighted avg       0.74      0.71      0.72       300

[[157  30]
 [ 57  56]]
315

with undersampling
Counter({1: 486, 2: 114})
              precision    recall  f1-score   support

        good       0.85      0.81      0.83       214
         bad       0.57      0.64      0.60        86

    accuracy                           0.76       300
   macro avg       0.71      0.72      0.72       300
weighted avg       0.77      0.76      0.76       300

[[173  31]
 [ 41  55]]
236

with oversampling
Counter({1: 586, 2: 214})
              precision    recall  f1-score   support

        good       0.83      0.77      0.80       214
         bad       0.51      0.62      0.56      

In all three classifiers we notice a decrease in the cost, compared to not sampling. Specifically, in Random Forest we achieve the smallest cost with Undersampling (107 compared to the initial which is 169) but also with combination. In SVM the smallest cost is achieved with combination (111 compared to the initial which is 180). Last but not least, for the Naive Bayes classifier the best cost is achieved, once again, by combination (227 compared to the initial which is 315). Overall, it is obvious that the costs of a false positive - incorrectly saying an applicant is a good credit risk, is higher than the cost of a false negative - incorrectly saying an applicant is a bad credit risk. Our best strategy, in general for all classifiers here is to use combination.

(we note that besides the decrease in cost, judging from f1 score, the Random Forest and Naive Bayes classifier get the best results in f1 scores, especially Naive Bayes, once again, using combination of oversampling and undersampling, which suits well the the additional reduction of the cost.)

#Sample Weights

##Random Forest

In [45]:
clf = RandomForestClassifier(n_estimators=10, random_state=0)

print("without weights")
model = clf.fit(X_train, y_train)
pred_test = model.predict(X_test)
print(classification_report(y_test, pred_test, target_names=target_names))
conf_m = confusion_matrix(y_test, pred_test).T 
print(conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)

print("\nwith weights")
weights = np.zeros(y_train.shape[0])
weights[np.where(y_train == 1)] = 5;
weights[np.where(y_train == 2)] = 1;
model = clf.fit(X_train, y_train, weights)
pred_test = clf.predict(X_test)
print(classification_report(y_test, pred_test, target_names=target_names))
conf_m = confusion_matrix(y_test, pred_test).T 
print(conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)

print("\nwith weights (alternative)")
clf.class_weight = {1: 5, 2: 1}
model = clf.fit(X_train, y_train)
pred_test = model.predict(X_test)
print(classification_report(y_test, pred_test, target_names=target_names))
conf_m = confusion_matrix(y_test, pred_test).T
print(conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)


without weights
              precision    recall  f1-score   support

        good       0.80      0.88      0.84       214
         bad       0.61      0.45      0.52        86

    accuracy                           0.76       300
   macro avg       0.71      0.67      0.68       300
weighted avg       0.75      0.76      0.75       300

[[189  47]
 [ 25  39]]
172


with weights
              precision    recall  f1-score   support

        good       0.78      0.86      0.82       214
         bad       0.54      0.41      0.46        86

    accuracy                           0.73       300
   macro avg       0.66      0.63      0.64       300
weighted avg       0.71      0.73      0.72       300

[[184  51]
 [ 30  35]]
201


with weights (alternative)
              precision    recall  f1-score   support

        good       0.78      0.86      0.82       214
         bad       0.54      0.41      0.46        86

    accuracy                           0.73       300
   macro avg  

##SVM

In [47]:
clf = SVC(kernel='linear')

print("without weights")
model = clf.fit(X_train, y_train)
pred_test = model.predict(X_test)
print(classification_report(y_test, pred_test, target_names=target_names))
conf_m = confusion_matrix(y_test, pred_test).T 
print('confusion matrix:',conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)

print("\nwith weights")
# Create the sample weights according to y
weights = np.zeros(y_train.shape[0])
weights[np.where(y_train == 1)] = 5;
weights[np.where(y_train == 2)] = 1;
model = clf.fit(X_train, y_train, weights)
pred_test = clf.predict(X_test)
print(classification_report(y_test, pred_test, target_names=target_names))
conf_m = confusion_matrix(y_test, pred_test).T 
print('confusion matrix:',conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)

print("\nwith weights (alternative)")
clf.class_weight = {1: 5, 2: 1}
model = clf.fit(X_train, y_train)
pred_test = model.predict(X_test)
print(classification_report(y_test, pred_test, target_names=target_names))
conf_m = confusion_matrix(y_test, pred_test).T
print('confusion matrix:',conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)


without weights
              precision    recall  f1-score   support

        good       0.82      0.87      0.85       214
         bad       0.62      0.53      0.57        86

    accuracy                           0.77       300
   macro avg       0.72      0.70      0.71       300
weighted avg       0.77      0.77      0.77       300

confusion matrix: [[186  40]
 [ 28  46]]
180


with weights
              precision    recall  f1-score   support

        good       0.71      1.00      0.83       214
         bad       0.00      0.00      0.00        86

    accuracy                           0.71       300
   macro avg       0.36      0.50      0.42       300
weighted avg       0.51      0.71      0.59       300

confusion matrix: [[214  86]
 [  0   0]]
86


with weights (alternative)
              precision    recall  f1-score   support

        good       0.71      1.00      0.83       214
         bad       0.00      0.00      0.00        86

    accuracy                     

##Naive Bayes

In [48]:
clf = GaussianNB()

print("without weights")
model = clf.fit(X_train, y_train)
pred_test = model.predict(X_test)
print(classification_report(y_test, pred_test, target_names=target_names))
conf_m = confusion_matrix(y_test, pred_test).T
print('confusion matrix:',conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)

print("\nwith weights")
# now create the sample weights according to y
weights = np.zeros(y_train.shape[0])
weights[np.where(y_train == 1)] = 5;
weights[np.where(y_train == 2)] = 1;
model = clf.fit(X_train, y_train, weights)
pred_test = clf.predict(X_test)
print(classification_report(y_test, pred_test, target_names=target_names))
conf_m = confusion_matrix(y_test, pred_test).T 
print('confusion matrix:',conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)

print("\nwith weights (alternative)")
clf.class_weight = {1: 5, 2: 1}
model = clf.fit(X_train, y_train)
pred_test = model.predict(X_test)
print(classification_report(y_test, pred_test, target_names=target_names))
conf_m = confusion_matrix(y_test, pred_test).T
print('confusion matrix:',conf_m)
loss = np.sum(conf_m * cost_m)
print("%d\n" %loss)


without weights
              precision    recall  f1-score   support

        good       0.84      0.73      0.78       214
         bad       0.50      0.65      0.56        86

    accuracy                           0.71       300
   macro avg       0.67      0.69      0.67       300
weighted avg       0.74      0.71      0.72       300

confusion matrix: [[157  30]
 [ 57  56]]
315


with weights
              precision    recall  f1-score   support

        good       0.79      0.89      0.84       214
         bad       0.60      0.41      0.49        86

    accuracy                           0.75       300
   macro avg       0.70      0.65      0.66       300
weighted avg       0.74      0.75      0.74       300

confusion matrix: [[191  51]
 [ 23  35]]
166


with weights (alternative)
              precision    recall  f1-score   support

        good       0.84      0.73      0.78       214
         bad       0.50      0.65      0.56        86

    accuracy                    

By applying weights, in general, we also get decreases in cost. In SVM we droped from 180 to 86 and with Naive Bayes, from 315 we droped to 166. Both of these cases worked well.
However, in Random Forest, not only did we not have a decrease, but instead, we got an even higher cost!

Judging from our classification reports, Naive Bayes, besides reducing the cost greatly, it has a very good performance on the metrics (eg. recall & f1).

#Minimizing Expected Cost

##Random Forest

In [49]:
# Train and evaluate a Random Forest classifier without cost minimization
clf = RandomForestClassifier(n_estimators=100, random_state=0)
print("No cost minimization")
model = clf.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T 
print(conf_m) 
print(np.sum(conf_m * cost_m))

print("\ncost minimization without probability calibration")
model = clf.fit(X_train, y_train)
y_pred_prob = model.predict_proba(X_test)
y_pred = np.argmin(np.matmul(y_pred_prob, np.array(cost_m).T), axis=1)+1
print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T
print(conf_m) 
print(np.sum(conf_m * cost_m))

# Train and evaluate a Random Forest classifier with cost minimization (sigmoid calibration)
print("\nCost minimization with sigmoid calibration")
cc = CalibratedClassifierCV(clf, method="sigmoid", cv=3)
model = cc.fit(X_train, y_train)
y_pred_prob = model.predict_proba(X_test)
y_pred = np.argmin(np.matmul(y_pred_prob, np.array(cost_m).T), axis=1)+1
print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T
print(conf_m) 
print(np.sum(conf_m * cost_m))

print("\ncost minimization with isotonic calibration")
cc = CalibratedClassifierCV(clf, method="isotonic", cv=3)
model = cc.fit(X_train, y_train)
y_pred_prob = model.predict_proba(X_test)
y_pred = np.argmin(np.matmul(y_pred_prob, np.array(cost_m).T), axis=1)+1
print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T
print(conf_m) 
print(np.sum(conf_m * cost_m))

No cost minimization
              precision    recall  f1-score   support

        good       0.79      0.89      0.84       214
         bad       0.61      0.43      0.50        86

    accuracy                           0.76       300
   macro avg       0.70      0.66      0.67       300
weighted avg       0.74      0.76      0.74       300

[[190  49]
 [ 24  37]]
169

cost minimization without probability calibration
              precision    recall  f1-score   support

        good       0.72      1.00      0.84       214
         bad       0.75      0.03      0.07        86

    accuracy                           0.72       300
   macro avg       0.73      0.52      0.45       300
weighted avg       0.73      0.72      0.61       300

[[213  83]
 [  1   3]]
88

Cost minimization with sigmoid calibration
              precision    recall  f1-score   support

        good       0.73      0.99      0.84       214
         bad       0.78      0.08      0.15        86

    accuracy 

##SVM

In [50]:
# Train and evaluate an SVM classifier without cost minimization
clf = SVC(kernel='linear', probability=True)
print("No cost minimization")
model = clf.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T 
print(conf_m) 
print(np.sum(conf_m * cost_m))

print("\ncost minimization without probability calibration")
model = clf.fit(X_train, y_train)
y_pred_prob = model.predict_proba(X_test)
y_pred = np.argmin(np.matmul(y_pred_prob, np.array(cost_m).T), axis=1)+1
print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T
print(conf_m) 
print(np.sum(conf_m * cost_m))

# Train and evaluate an SVM classifier with cost minimization (sigmoid calibration)
print("\nCost minimization with sigmoid calibration")
cc = CalibratedClassifierCV(clf, method="sigmoid", cv=3)
model = cc.fit(X_train, y_train)
y_pred_prob = model.predict_proba(X_test)
y_pred = np.argmin(np.matmul(y_pred_prob, np.array(cost_m).T), axis=1)+1
print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T
print(conf_m) 
print(np.sum(conf_m * cost_m))

print("\ncost minimization with isotonic calibration")
cc = CalibratedClassifierCV(clf, method="isotonic", cv=3)
model = cc.fit(X_train, y_train)
y_pred_prob = model.predict_proba(X_test)
y_pred = np.argmin(np.matmul(y_pred_prob, np.array(cost_m).T), axis=1)+1
print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T
print(conf_m) 
print(np.sum(conf_m * cost_m))

No cost minimization
              precision    recall  f1-score   support

        good       0.82      0.87      0.85       214
         bad       0.62      0.53      0.57        86

    accuracy                           0.77       300
   macro avg       0.72      0.70      0.71       300
weighted avg       0.77      0.77      0.77       300

[[186  40]
 [ 28  46]]
180

cost minimization without probability calibration
              precision    recall  f1-score   support

        good       0.72      0.98      0.83       214
         bad       0.50      0.05      0.09        86

    accuracy                           0.71       300
   macro avg       0.61      0.51      0.46       300
weighted avg       0.66      0.71      0.62       300

[[210  82]
 [  4   4]]
102

Cost minimization with sigmoid calibration
              precision    recall  f1-score   support

        good       0.72      0.99      0.83       214
         bad       0.50      0.03      0.07        86

    accuracy

##Naive Bayes

In [51]:
# Train and evaluate a Naive Bayes classifier without cost minimization
clf = GaussianNB()
print("No cost minimization")
model = clf.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T 
print(conf_m) 
print(np.sum(conf_m * cost_m))

print("\ncost minimization without probability calibration")
model = clf.fit(X_train, y_train)
y_pred_prob = model.predict_proba(X_test)
y_pred = np.argmin(np.matmul(y_pred_prob, np.array(cost_m).T), axis=1)+1
print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T
print(conf_m) 
print(np.sum(conf_m * cost_m))

# Train and evaluate a Naive Bayes classifier with cost minimization (sigmoid calibration)
print("\nCost minimization with sigmoid calibration")
cc = CalibratedClassifierCV(clf, method="sigmoid", cv=3)
model = cc.fit(X_train, y_train)
y_pred_prob = model.predict_proba(X_test)
y_pred = np.argmin(np.matmul(y_pred_prob, np.array(cost_m).T), axis=1)+1
print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T
print(conf_m) 
print(np.sum(conf_m * cost_m))

print("\ncost minimization with isotonic calibration")
cc = CalibratedClassifierCV(clf, method="isotonic", cv=3)
model = cc.fit(X_train, y_train)
y_pred_prob = model.predict_proba(X_test)
y_pred = np.argmin(np.matmul(y_pred_prob, np.array(cost_m).T), axis=1)+1
print(classification_report(y_test, y_pred, target_names=target_names))
conf_m = confusion_matrix(y_test, y_pred).T
print(conf_m) 
print(np.sum(conf_m * cost_m))

No cost minimization
              precision    recall  f1-score   support

        good       0.84      0.73      0.78       214
         bad       0.50      0.65      0.56        86

    accuracy                           0.71       300
   macro avg       0.67      0.69      0.67       300
weighted avg       0.74      0.71      0.72       300

[[157  30]
 [ 57  56]]
315

cost minimization without probability calibration
              precision    recall  f1-score   support

        good       0.79      0.89      0.84       214
         bad       0.60      0.41      0.49        86

    accuracy                           0.75       300
   macro avg       0.70      0.65      0.66       300
weighted avg       0.74      0.75      0.74       300

[[191  51]
 [ 23  35]]
166

Cost minimization with sigmoid calibration
              precision    recall  f1-score   support

        good       0.71      1.00      0.83       214
         bad       0.00      0.00      0.00        86

    accuracy

Here, we achieved a cost minimization in all three cases. In Random Forest the smallest cost was achieved in cost minimization without probability calibration. In SVM the smaller costs are noticed in cost minimization without probability calibration and sigmoid and lastly in Naive Bayes, cost minimization with sigmoid calibration has achieved also the smallest cost.
Overall, calibration "saves the case" in Naive Bayes and in SVM, specifically with the sigmoid calibration only. In Random Forest we prefer cost minimization without probability calibration, while the isotonic calibration in SVM does not give the best results, while we note that the metrics of our classification report are the best in Naive Bayes, which also has the greatest minimization in the cost by using (especially sigmoid) calibration.